In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_fixed
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams, clfName):
    path = ''#"old_results/batch/dynamic/gridsearch_amanda_dynamic_EVL-{}.txt".format(clfName)
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main():
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    #datasets = [setup.load2CHT, setup.load4CRE_V1, 
    #            setup.load4CRE_V2, setup.loadCSurr, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
    #            setup.loadFG_2C_2D, setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    batches=100
    datasets=[setup.loadElecData]
    #datasets=[setup.loadKeystroke]
    #batches=8
    #datasets=[setup.loadNOAADataset]
    #batches=50
    
    arrClfName = ['LP','KNN']#['SGD', 'NB', 'RF', 'LP', 'KNN']
    for clfName in arrClfName:
        print("**************** BEGIN of {} results ****************".format(clfName))
        poolSize = None
        isBatchMode = True
        #testing grid search
        for i in range(len(datasets)):
            '''batches=100
            if i==len(datasets)-2:
                batches=50
            elif i==len(datasets)-1:
                batches=8
            '''    
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)
            
            initialLabeledData = int(0.05*len(dataLabels))
            sizeOfBatch = int((len(dataLabels)-initialLabeledData)/batches)

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))
            #"excludingPercentage" : [0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.6, 0.55, 0.5],
            tuned_params = [{"sizeOfBatch":[sizeOfBatch], "batches":[batches], "poolSize":[poolSize],
                             "isBatchMode":[isBatchMode], "initialLabeledData":[initialLabeledData], "clfName":[clfName]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                #gs = grid_selection_amanda_fixed.run(**g)
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(dataValues, dataLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    raise Exception

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            #writeResults(description, finalScore, best_grid, clfName)
        print("******** END of {} results ********".format(clfName))
    
if __name__ == "__main__":
    main()

'''
## Fixed
#keyboard dataset
89.14625
{'K': 2, 'batches': 8, 'clfName': 'LP', 'excludingPercentage': 0.85, 'initialLabeledData': 80, 'isBatchMode': True, 
'poolSize': None, 'sizeOfBatch': 190}
90.1325
{'K': 7, 'batches': 8, 'clfName': 'KNN', 'excludingPercentage': 0.8, 'initialLabeledData': 80, 'isBatchMode': True, 
'poolSize': None, 'sizeOfBatch': 190}
#electricity dataset
73.7737
{'K': 11, 'batches': 100, 'clfName': 'LP', 'excludingPercentage': 0.6, 'initialLabeledData': 1377, 'isBatchMode': True, 
'poolSize': None, 'sizeOfBatch': 261}
73.8808
{'K': 6, 'batches': 100, 'clfName': 'KNN', 'excludingPercentage': 0.6, 'initialLabeledData': 1377, 'isBatchMode': True, 
'poolSize': None, 'sizeOfBatch': 261}
#NOAA dataset
71.7032
{'K': 2, 'batches': 50, 'clfName': 'LP', 'excludingPercentage': 0.5, 'initialLabeledData': 907, 'isBatchMode': True, 
'poolSize': None, 'sizeOfBatch': 345}
76.3758
{'K': 8, 'batches': 50, 'clfName': 'KNN', 'excludingPercentage': 0.7, 'initialLabeledData': 907, 'isBatchMode': True, 
'poolSize': None, 'sizeOfBatch': 345}

## DYNAMIC
#Keyboard
89.27625
{'K': 2, 'batches': 8, 'clfName': 'LP', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 190}
90.6575
{'K': 4, 'batches': 8, 'clfName': 'KNN', 'initialLabeledData': 80, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 190}
#Electricity
66.8198
{'K': 11, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 261}
65.5592
{'K': 12, 'batches': 100, 'clfName': 'KNN', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 261}
#NOAA
71.7032
{'K': 2, 'batches': 50, 'clfName': 'LP', 'initialLabeledData': 907, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 345}
74.2884
{'K': 6, 'batches': 50, 'clfName': 'KNN', 'initialLabeledData': 907, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 345}
'''

'''
# Small modification in the formula of automatic cutting ( [sqrt(2)-avg_helling_dist]/sqrt(2) )
## DYNAMIC
#Keyboard
'''
'''
testing
Electricity data. 7 features. 2 classes.: 100 batches of 261 instances
64.0607 {'K': 2, 
56.0269 {'K': 3,
65.2794 {'K': 4, 
65.3871 {'K': 5,
66.4708 {'K': 6, 
66.3908 {'K': 7, 
66.5555 {'K': 8, 

**************** BEGIN of LP results ****************
Electricity data. 7 features. 2 classes.: 100 batches of 261 instances
64.0607 {'K': 2, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 261}
56.0269 {'K': 3, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 261}
65.2794 {'K': 4, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 261}
65.3871 {'K': 5, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 261}
66.4708 {'K': 6, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 261}
66.3908 {'K': 7, 'batches': 100, 'clfName': 'LP', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 261}
66.5555 {'K': 8, 'batches': 100, 'clfName': 'LP', 'in

KeyboardInterrupt: 